In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import nltk

/projects/nanopore-working/rojin/nanoraw-signalAlign-nanopolish/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
embedding = pd.read_table('glove.twitter.27B.25d.txt', sep = " ", header= None)

In [3]:
embedding = embedding.dropna()

In [4]:
len(embedding[0])

1193508

In [5]:
embedding_matrix = embedding[list(range(1,26))].as_matrix()

In [6]:
# editing the input sentences
training_set = pd.read_csv('train.csv', encoding="ISO-8859-1")
training_set.head(3)

,handle,tweet
0,realDonaldTrump,"In Tampa, Florida- thank you to all of our out..."
1,realDonaldTrump,Poll: @realDonaldTrump vs. @HillaryClinton amo...
2,HillaryClinton,Obama on whether Trump could be trusted with U...


In [7]:
training_set['handle']=training_set['handle'].map({'HillaryClinton':1,'realDonaldTrump':0})

In [8]:
training_set.head(3)

,handle,tweet
0,0,"In Tampa, Florida- thank you to all of our out..."
1,0,Poll: @realDonaldTrump vs. @HillaryClinton amo...
2,1,Obama on whether Trump could be trusted with U...


In [9]:
# I adapted this from another a site, need to cite it here
def remove_by_regex(tweets, regexp):
    tweets.loc[:, "tweet"].replace(regexp, "", inplace=True)
    return tweets

def remove_special_chars(tweets):  # it unrolls the hashtags to normal words
    for remove in map(lambda r: re.compile(re.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                 "@", "%", "^", "*", "(", ")", "{", "}",
                                                                 "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                 "!", "?", ".", "'",
                                                                 "--", "---", "#", "\n" , "—@", "→"]):
        tweets.loc[:, "tweet"].replace(remove, "", inplace=True)
    return tweets

def remove_things(tweets):
    remove_by_regex(tweets, re.compile(r"http.?://[^\s]+[\s]?"))
    remove_by_regex(tweets, re.compile(r"@[^\s]+[\s]?"))
    remove_by_regex(tweets, re.compile(r"\s?[0-9]+\.?[0-9]*"))
    remove_special_chars(tweets)
    return tweets

In [10]:
remove_things(training_set).head(3)

,handle,tweet
0,0,In Tampa Florida thank you to all of our outst...
1,0,Poll vs among white Evangelicals
2,1,Obama on whether Trump could be trusted with U...


In [11]:
#tokenize
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize
def tok(tweets):
    p = []
    for i in tweets['tweet']:
        tokenizer = RegexpTokenizer(r'\w+')
        p.append(tokenizer.tokenize(i))
    tweets['tok_tweet'] = p
    return tweets

In [12]:
tok(training_set).head(3)

,handle,tweet,tok_tweet
0,0,In Tampa Florida thank you to all of our outst...,"[In, Tampa, Florida, thank, you, to, all, of, ..."
1,0,Poll vs among white Evangelicals,"[Poll, vs, among, white, Evangelicals]"
2,1,Obama on whether Trump could be trusted with U...,"[Obama, on, whether, Trump, could, be, trusted..."


In [13]:
#stemmer
sno = nltk.stem.SnowballStemmer('english')
def stemm(tweets):
    p = []
    for i in tweets['tok_tweet']:
        q = []
        for j in i:
            q.append(sno.stem(j))
        p.append(q)
    tweets['tok_tweet'] = p
    return tweets

In [14]:
stemm(training_set).head(3)

,handle,tweet,tok_tweet
0,0,In Tampa Florida thank you to all of our outst...,"[in, tampa, florida, thank, you, to, all, of, ..."
1,0,Poll vs among white Evangelicals,"[poll, vs, among, white, evangel]"
2,1,Obama on whether Trump could be trusted with U...,"[obama, on, whether, trump, could, be, trust, ..."


In [15]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /soe/rojinsafavi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords
def stop_words_eliminatin(tweets):
    p = []
    stop_words = set(stopwords.words('english'))
    for i in tweets['tok_tweet']:
        q = []
        for j in i:
            if j not in stop_words:
                q.append(j)
        p.append(q)
        
    tweets['tok_tweet'] = p
    return tweets

In [17]:
stop_words_eliminatin(training_set).head(3)

,handle,tweet,tok_tweet
0,0,In Tampa Florida thank you to all of our outst...,"[tampa, florida, thank, outstand, volunt, want..."
1,0,Poll vs among white Evangelicals,"[poll, vs, among, white, evangel]"
2,1,Obama on whether Trump could be trusted with U...,"[obama, whether, trump, could, trust, us, nucl..."


In [18]:
# add tokenized sentece length:
P = []
for i in training_set['tok_tweet']:
    length = len(i)
    P.append(length)
training_set['length'] = P

In [19]:
# number of words in our dict:
count = {}
for i in training_set['tok_tweet']:
    for j in i:
        if j not in count:
            count[j] = 1
        else:
            count[j] += 1
len(count)

6444

In [20]:
# find the index of words for the sentence from the embedding
word_indexes = {}

for i,j in enumerate(embedding[0]):
    if j not in word_indexes:
        word_indexes[str(j)] = i
    
for i,j in enumerate(count.keys()):
    if j not in word_indexes:
        word_indexes[str(j)] = len(word_indexes)

In [21]:
k = []
for i in training_set['tok_tweet']:
    l = []
    for j in i:
        l.append(word_indexes[j])
    k.append(l)
training_set['word_ids'] = k

training_set = training_set[training_set['word_ids'].apply(lambda c: c!=[])] #removing empty tweeets

In [22]:
training_set.head(3)

,handle,tweet,tok_tweet,length,word_ids
0,0,In Tampa Florida thank you to all of our outst...,"[tampa, florida, thank, outstand, volunt, want...",7,"[14387, 3780, 336, 1193508, 668240, 128, 1193509]"
1,0,Poll vs among white Evangelicals,"[poll, vs, among, white, evangel]",5,"[9836, 916, 8634, 811, 469675]"
2,1,Obama on whether Trump could be trusted with U...,"[obama, whether, trump, could, trust, us, nucl...",10,"[1380, 4527, 28622, 296, 978, 290, 11639, 1604..."


In [23]:
#split into training and validation

from sklearn.model_selection import train_test_split

train, test = train_test_split(training_set, test_size=0.2)

In [24]:
print(len(train))
print(len(test))

3993
999


In [25]:
train.head(3)

,handle,tweet,tok_tweet,length,word_ids
3562,1,We have some questions about this letter from ...,"[question, letter, donald, trump, doctor]",5,"[1410, 4735, 14348, 28622, 3056]"
2220,0,Nevada we are on our way VoteTrumpNV TrumpCauc...,"[nevada, way, votetrumpnv, trumpcaucus, locat]",5,"[28283, 270, 1194098, 1194735, 563445]"
2575,0,Thank you South Bend Indiana Everyone get out ...,"[thank, south, bend, indiana, everyon, get, am...",10,"[336, 2347, 14817, 11649, 125834, 86, 12797, 1..."


In [27]:
test.head(3)

,handle,tweet,tok_tweet,length,word_ids
1130,0,WSJNBC Poll Donald Trump Widens His Lead in Re...,"[wsjnbc, poll, donald, trump, widen, lead, rep...",10,"[1194189, 9836, 14348, 28622, 108369, 1899, 16..."
1879,1,Secretary Clinton has won the Democratic nomin...,"[secretari, clinton, democrat, nomin, process,...",7,"[596329, 13119, 34127, 523024, 7011, 1121792, ..."
4825,0,Chris thank you so much for your wonderful en...,"[chris, thank, much, wonder, endors, let]",6,"[1668, 336, 201, 1112, 115503, 264]"


We’re going to build an RNN that accepts batches of data from the “word_ids” column and predicts the “handle”. The first step is to construct a simple iterator that returns batches of inputs along with their targets, and the length of each input:

In [28]:
class SimpleDataIterator():
    def __init__(self, df):
        self.df = df
        self.size = len(self.df)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n-1 > self.size:
            self.epochs += 1
            self.shuffle()
        res = self.df.ix[self.cursor:self.cursor+n-1]
        self.cursor += n
        return res['word_ids'], res['handle'], res['length']

In [29]:
data = SimpleDataIterator(training_set)
d = data.next_batch(3)
print('Input sequences\n', d[0], end='\n\n')
print('Target values\n', d[1], end='\n\n')
print('Sequence lengths\n', d[2])

Input sequences
 0          [1151260, 1214, 3780, 287, 234, 336, 29892]
1    [410, 434, 12341, 16190, 523024, 101980, 185, ...
2    [336, 62960, 2610, 44446, 2705, 25732, 1112, 3...
Name: word_ids, dtype: object

Target values
 0    0
1    1
2    0
Name: handle, dtype: int64

Sequence lengths
 0     7
1    14
2     8
Name: length, dtype: int64


/projects/nanopore-working/rojin/nanoraw-signalAlign-nanopolish/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [30]:
#padding
class PaddedDataIterator(SimpleDataIterator):
    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            self.shuffle()
        res = self.df.ix[self.cursor:self.cursor+n-1]
        self.cursor += n

        # Pad sequences with 0s so they are all the same length
        maxlen = max(res['length'])
        x = np.zeros([n, maxlen], dtype=np.int32)
        for i, x_i in enumerate(x):
            x_i[:res['length'].values[i]] = res['word_ids'].values[i]

        return x, res['handle'], res['length']

In [31]:
data = PaddedDataIterator(training_set)
d = data.next_batch(3)
print('Input sequences\n', d[0], end='\n\n')

Input sequences
 [[   182   1937   2183      0      0      0      0      0      0      0
       0      0      0      0]
 [  1368    110 213128   2412 224411  12750    409 790821  42569  11639
    7872    201  62960  91909]
 [  3132  39468  49653  66349    620   7011   1590  28622      0      0
       0      0      0      0]]



/projects/nanopore-working/rojin/nanoraw-signalAlign-nanopolish/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [32]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def build_graph(
    vocab_size = len(word_indexes),
    state_size = 64,
    batch_size = 256,
    num_classes = 2):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
    keep_prob = tf.placeholder_with_default(1.0, [])

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    cell = tf.nn.rnn_cell.GRUCell(state_size)
    init_state = tf.get_variable('init_state', [1, state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen,
                                                 initial_state=init_state)

    # Add dropout, as the model otherwise quickly overfits
    rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)

    """
    Obtain the last relevant output. The best approach in the future will be to use:

        last_rnn_output = tf.gather_nd(rnn_outputs, tf.pack([tf.range(batch_size), seqlen-1], axis=1))

    which is the Tensorflow equivalent of numpy's rnn_outputs[range(30), seqlen-1, :], but the
    gradient for this op has not been implemented as of this writing.

    The below solution works, but throws a UserWarning re: the gradient.
    """
    idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)

    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
        'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }

In [33]:
g = build_graph()

/projects/nanopore-working/rojin/nanoraw-signalAlign-nanopolish/anaconda2/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [34]:
def train_graph(graph, batch_size = 256, num_epochs = 10, iterator = PaddedDataIterator):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        tr = iterator(train)
        te = iterator(test)

        step, accuracy = 0, 0
        tr_losses, te_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: 0.6}
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                te_epoch = te.epochs
                while te.epochs == te_epoch:
                    step += 1
                    batch = te.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_

                te_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- te:", te_losses[-1])

    return tr_losses, te_losses

In [35]:
tr_losses, te_losses = train_graph(g)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


/projects/nanopore-working/rojin/nanoraw-signalAlign-nanopolish/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


Accuracy after epoch 1  - tr: 0.53955078125 - te: 0.5625
Accuracy after epoch 2  - tr: 0.6174479166666667 - te: 0.6080729166666666
Accuracy after epoch 3  - tr: 0.6653645833333334 - te: 0.6419270833333334
Accuracy after epoch 4  - tr: 0.7158854166666667 - te: 0.6888020833333334
Accuracy after epoch 5  - tr: 0.76484375 - te: 0.7356770833333334
Accuracy after epoch 6  - tr: 0.8059895833333334 - te: 0.7799479166666666
Accuracy after epoch 7  - tr: 0.8473958333333333 - te: 0.83203125
Accuracy after epoch 8  - tr: 0.8763020833333334 - te: 0.8359375
Accuracy after epoch 9  - tr: 0.8986979166666667 - te: 0.86328125
Accuracy after epoch 10  - tr: 0.91484375 - te: 0.8580729166666666
